In [1]:
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import balanced_accuracy_score
from scipy.stats import uniform, randint, loguniform, norm

# Pre-processing

In [2]:
# Read the data
data = pd.read_csv('C:/Users/vabalagon/Desktop/Meta/New Workflow/data/2 data for modeling (With PCA).csv')

# Get the features and target variable from the dataframe
X = data.drop(['Survey ID', 'Response Date', 'Likelihood to Recommend'], axis=1).to_numpy()
y = data['Likelihood to Recommend'].to_numpy()

# Split the data into test and train sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.copy(), y, test_size = 0.25, shuffle=True, random_state=42) #, stratify=y_smote

# Apply SMOTE oversampling to the TRAINING SET ONLY
sm = SMOTE(random_state=42)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train_smote))

Resampled dataset shape Counter({0: 8116, 2: 8116, 1: 8116})


# XGBoost

In [3]:
import xgboost as xgb

C:\Users\vabalagon\.conda\envs\DataScienceStandard\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
from sklearn.utils import class_weight

classes_weights = class_weight.compute_sample_weight(
    class_weight='balanced',
    y=y_train_smote
)

In [5]:
# xgb_model_clf = xgb.XGBClassifier(objective="multi:softmax",
#                                   eval_metric='mlogloss',
#                                   use_label_encoder=False,
#                                  )

# params = {
#     "n_estimators": randint(2, 300), # default 100
#     "colsample_bytree": uniform(0.1, 1),
#     "gamma": uniform(0.1, 1),
#     "max_depth": randint(2, 7), # default 3
#     "subsample": uniform(0.1, 1)
# }

# xgb_model = RandomizedSearchCV(xgb_model_clf, 
#                             param_distributions=params, 
#                             random_state=42, 
#                             n_iter=200, 
#                             cv=5, 
#                             verbose=5, 
#                             n_jobs=-1, 
#                             return_train_score=True,
#                             scoring='balanced_accuracy')

# xgb_model.fit(X_train_smote, y_train_smote, sample_weight=classes_weights)

# print("\nBest parameter:", xgb_model.best_params_)
# print("Training set cross-validation balanced accuracy score:", xgb_model.best_score_) 
# print("Test set balanced accuracy score:", balanced_accuracy_score(y_test, xgb_model.predict(X_test)))

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


C:\Users\vabalagon\.conda\envs\DataScienceStandard\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
215 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vabalagon\.conda\envs\DataScienceStandard\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vabalagon\.conda\envs\DataScienceStandard\lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
  File "C:\Users\vabalagon\.conda\envs\DataScienceStandard\lib\site-packages\xgboost\sklearn.py", line 1250, in fit
 


Best parameter: {'colsample_bytree': 0.7955160864261275, 'gamma': 0.2393314544058757, 'max_depth': 6, 'n_estimators': 144, 'subsample': 0.8555511385430487}
Training set cross-validation balanced accuracy score: 0.88491362869749
Test set balanced accuracy score: 0.7616455441270329



Best parameter: {'colsample_bytree': 0.7955160864261275, 'gamma': 0.2393314544058757, 'max_depth': 6, 'n_estimators': 144, 'subsample': 0.8555511385430487}
Training set cross-validation balanced accuracy score: 0.88491362869749
Test set balanced accuracy score: 0.7616455441270329

# Final model

In [11]:
xgb_final = xgb.XGBClassifier(objective="multi:softmax",
                              n_estimators= 144,
                              max_depth= 3,
                              gamma= 0.2393314,
                              booster='gbtree',
                              subsample=0.8555511385430487,
                              colsample_bytree =0.7955160864261275,
                                eval_metric='mlogloss',
                                use_label_encoder=False)

xgb_final.fit(X_train_smote, y_train_smote, sample_weight=classes_weights)

print("Test set balanced accuracy score:", balanced_accuracy_score(y_test, xgb_final.predict(X_test)))

Test set balanced accuracy score: 0.7743044371605187


##### Accuracy per class

In [12]:
y_pred = xgb_final.predict(X_test)

for y_i in np.unique(y_test)[::-1]:    
    # Find the indices of y_i in the true labels
    indices_i = np.where(y_test == y_i)
    
    # Computes the accuracy
    print('class', y_i, 'Accuracy: ', str(round(np.sum(y_test[indices_i] == y_pred[indices_i])/ len(np.where(y_test==y_i)[0]), 3)))

class 2 Accuracy:  0.951
class 1 Accuracy:  0.493
class 0 Accuracy:  0.879


# Normal distribution

In [ ]:
norm.rvs(0, .1, 10)

In [ ]:
loc = 0
scale = 1
plt.figure()
plt.hist(norm.rvs(loc, scale, 1000000), bins=50)

In [ ]:
plt.figure()
plt.hist(uniform.rvs(0, 0.5, 100000))

# Uniform distribution

In [ ]:
plt.figure()
plt.hist(uniform.rvs(1, 20, 100000))

# Log uniform distribution

In [ ]:
plt.figure()
plt.hist(loguniform.rvs(0.1, 10, size=10000))